In [79]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentil

from unicodedata import normalize

In [62]:
avisos_detalle = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalle2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalle = pd.merge(avisos_detalle,avisos_detalle2,how='outer')

avisos_detalle2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')

avisos_detalle = pd.merge(avisos_detalle,avisos_detalle2,how='outer')

avisos_detalle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25563 entries, 0 to 25562
Data columns (total 11 columns):
idaviso                 25563 non-null int64
idpais                  25563 non-null int64
titulo                  25563 non-null object
descripcion             25563 non-null object
nombre_zona             25563 non-null object
ciudad                  137 non-null object
mapacalle               1854 non-null object
tipo_de_trabajo         25563 non-null object
nivel_laboral           25563 non-null object
nombre_area             25563 non-null object
denominacion_empresa    25556 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


In [63]:
avisos_detalle_txt = avisos_detalle['descripcion']

In [64]:
avisos_detalle_txt.sample(50)

7672     <p>Maison de la Parfumerie S.A. seleccionará p...
10442    <p><span style=""><span style="">Accenture es ...
3235     <p>Buscamos Técnico Electrtónico Especializado...
1633     <p>Nos encontramos en busca de personal para A...
11100    <p style=""><span style="">Para importante emp...
10919    <p>Obra Social Unión Personal se encuentra en ...
2427     <p>Importante empresa del rubro Autopartista s...
9177     <p><strong>Para importante empresa del sector ...
4545     <p>Importante Estudio Internacional de Arquite...
15488    <p><strong>SOLUTIX</strong> está en búsqueda d...
10165    <p>Swiss Medical Group incorporará para su uni...
6018     <p style=""><strong><span style="">IT Resource...
11014    <p style="">Importante empresa del rubro plást...
15146    <p><em><strong>GRUPO ITALIA CONSTRUCCIONES Y P...
12207    <p style="">Nos encontramos en la búsqueda de ...
14381    <p>Somos un importante grupo de Concesionarios...
16937    <div style=""><ul style=""><li style=""><em><s.

In [65]:
def textClean(text):
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", text), 0, re.I)
    text = normalize( 'NFC', text)
    text = re.sub(r"[,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stops = {'para', 'de', 'en', 'presentarse', 'entrevista', 
             'y', 'a','trabajar','o','as','zona','te','estamos','al'
            'buscando','del','importante','hs','horario','tu','como',
            'al','con','por','nueva','la','e','p>','li>','ul>','strong',
            'style','p','span'}
    text = [w for w in text if not w in stops]    
    text = " ".join(text)
    text = text.replace("."," ").replace(","," ").replace("("," ").replace(")"," ")
    text = text.replace("/"," ").replace("-"," ").replace("!"," ").replace("¡"," ")
    text = text.replace("\""," ").replace("*"," ").replace(">"," ")
    return(text)

In [66]:
trainText = []
for it in avisos_detalle_txt:
    newT = textClean(it)
    trainText.append(newT)
    

In [67]:
trainText

['nuestra busqueda se orienta incorporacion vendedoras nuestro salon ventas articulos el hogar son requisitos   estudios secundarios completos   edad años   muy buena presencia   excelente trato   experiencia ventas  excluyente    proactividad  dedicacion full time',
 'ingeniero electricista electronico conocimientos mandos potencia baja media tension  interruptores seccionadores contactores normas electricas  variadores velocidad plcs ingles tecnico manejo pc entorno windows office contar experiencia promocion asistencia asesoramiento clientes productos automatizacion control industrial asi tambien el desarrollo documentacion el soporte fuerza venta se dara prioridad aquellos que acrediten experiencia previa puestos similares empresas rubro',
 'diseñadores graficos quark express revista grafica buscando profesionales diseño  edad entre años  altos conocimientos herramientas diseño grafico principalmente quark express las personas tendran bajo su responsabilidad edicion maquetacion dis

In [68]:
testText = trainText[18000:]

In [69]:
len(trainText)

25563

In [70]:
del(trainText[18000:])

In [71]:
len(trainText)

18000

In [72]:
len(testText)

7563

In [74]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5)
X_train = vectorizer.fit_transform(trainText)

In [76]:
X_test = vectorizer.transform(testText)

In [78]:
feature_names = vectorizer.get_feature_names()
feature_names

['________________________________________________________________',
 '__________________________________________________________________',
 '______________________________________________________________________________',
 '__________________________________________________________________________________________________',
 '_____________________________________________________________________________responsabilidadesd',
 '_alta',
 '_analisis',
 '_atencion',
 '_auditar',
 '_auditoria',
 '_aumentar',
 '_b',
 '_blank',
 '_carga',
 '_cobranzas',
 '_comunicacion',
 '_conciliacion',
 '_conocimiento',
 '_conocimientos',
 '_control',
 '_dinamismo',
 '_disponibilidad',
 '_edad',
 '_electricidad',
 '_electromecanico',
 '_emision',
 '_encontrar',
 '_experiencia',
 '_fidelizar',
 '_fuerte',
 '_fuertes',
 '_generar',
 '_gestion',
 '_gestionar',
 '_habilidades',
 '_horario',
 '_manejo',
 '_ofrecemos',
 '_optimizacion',
 '_orden',
 '_orientacion',
 '_preferentemente',
 '_proactivo',
 '_programa',
 

In [86]:
len(X_train.data)

1409930

In [44]:
    ch2 = SelectKBest(chi2)
    X_train = ch2.fit_transform(X_train, y_train)
    X_test = ch2.transform(X_test)
    if feature_names:
        # keep selected feature names
        feature_names = [feature_names[i] for i
                         in ch2.get_support(indices=True)]

IDF: [9.93115543 9.93115543 9.93115543 ... 9.93115543 8.83254314 9.93115543]


In [45]:
tf_idf_matrix = tfidf.transform(freq_term_matrix)
print (tf_idf_matrix.todense())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [49]:
tfidf.get_feature_names

AttributeError: 'TfidfTransformer' object has no attribute 'get_feature_names'